In [1]:
import mbuild as mb
from foyer import Forcefield
from mbuild.lib.recipes.polymer import Polymer
import unyt as u

import mosdef_cassandra as mc

/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/utils/decorators.py:134: Warning: freud_generate_bonds is an experimental feature and is not subject to follow standard deprecation cycles. Use at your own risk! 
  warn(


## Prepare recipes

### Exercise 1

In [2]:
class _CH2(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH2"
        # Create a partilce name _CH2
        _ch2 = mb.Compound(name="_CH2")
        self.add(_ch2, "_CH2")
        
        # Create ports to be attached to the _CH2 particles
        port_up = mb.Port(anchor=_ch2, orientation=[0, 0, 1], separation=0.07)
        port_down = mb.Port(anchor=_ch2, orientation=[0, 0, -1], separation=0.07)
        self.add(port_up, "up")
        self.add(port_down, "down")
        
class _CH3(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH3"
        
        # Create a particle name _CH3
        _ch3 = mb.Compound(name="_CH3")
        self.add(_ch3, "_CH3")
        
        # Create a port to be attached to the _CH3 particle
        port_up = mb.Port(anchor=_ch3, orientation=[0, 0, 1], separation=0.07)
        self.add(port_up, "up")

            

### Exercise 2 

In [3]:
class CG_Alkane(mb.Compound):
    def __init__(self, n):
        super().__init__()
        self.name = "CG_Alkane"
    
        if n == 1: 
            _ch4 = mb.Compound(name="_CH4")
            self.add(_ch4, "_CH4")
        elif n == 2:
            _ch3a = _CH3() 
            _ch3b = _CH3() 
            self.add(_ch3a, "_CH3_a")
            self.add(_ch3b, "_CH3_b")
            mb.force_overlap(move_this=_ch3a, 
                             from_positions=_ch3a["up"], 
                             to_positions=_ch3b["up"])
        else:
            _ch3top = _CH3()
            self.add(_ch3top, "link0")
            for i in range(n - 2):
                _ch2 = _CH2()
                mb.force_overlap(move_this=_ch2, 
                                 from_positions=_ch2["down"],
                                 to_positions=self[f"link{i}"]["up"])
                self.add(_ch2, f"link{i+1}")
                
            _ch3bot = _CH3()
            self.add(_ch3bot, f"link{i+2}")
            mb.force_overlap(move_this=_ch3bot, 
                             from_positions=_ch3bot["up"], 
                             to_positions=self[f"link{i+1}"]["up"])


In [4]:
from mbuild.lib.recipes import Polymer

cg_alkane = Polymer(monomers=[_CH2()], end_groups=[_CH3(), _CH3()])
cg_alkane.build(n=5)

### Exercise 3

In [5]:
n_pentane = CG_Alkane(5)
pentane_box = mb.fill_box(compound=n_pentane, 
                          n_compounds=150, 
                          box=[3.2, 3.2, 3.2],
                          seed=12345, 
                          edge=0.2)

/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([0. 0. 0.]), 0 bonds, id: 5915130704>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH2 pos=([ 9.6244e-18 -8.3696e-18  1.4000e-01]), 0 bonds, id: 5914860848>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/formats/xyz.py:68: UserWarning: No matching element found for EP; the particle will be added to the compound without an element attribute.
  warn(


### Exercise 4

In [6]:
import foyer 
from foyer import Forcefield

trappe = Forcefield(name="trappe-ua")
pentane_box.energy_minimize(forcefield="trappe-ua", 
                            steps=10**5)

pentane_box.visualize()


/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _CH4
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _CH2
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _HC
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([0.89   2.0663 2.5915]), 0 bonds, id: 5915185216>'; and no matching elements 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Exercise 5

In [7]:
# Load forcefields
trappe = Forcefield(name="trappe-ua")

# Use foyer to apply forcefields
typed_pentane = trappe.apply(n_pentane)
species_list = [typed_pentane]

# Create box and species list
box_list = [pentane_box]
mols_in_boxes = [[150]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moveset = mc.MoveSet("npt", species_list)

moveset.prob_volume = 0.010

thermo_props = [
    "energy_total",
    "energy_intervdw",
    "pressure",
    "volume",
    "nmols",
    "mass_density",
]

custom_args = {
    "run_name": "equil",
    "charge_style": "none",
    "rcut_min": 2.0 * u.angstrom,
    "vdw_cutoff": 14.0 * u.angstrom,
    "units": "sweeps",
    "steps_per_sweep": 450,
    "coord_freq": 50,
    "prop_freq": 10,
    "properties": thermo_props,
}


mc.run(
    system=system,
    moveset=moveset,
    run_type="equilibration",
    run_length=2400,
    pressure=14.02 * u.bar,
    temperature=372 * u.K,
    **custom_args,
)

Using the following executables for Cassandra:
Python: /Users/quachcd/mambaforge/envs/general/bin/python
library_setup: /Users/quachcd/mambaforge/envs/general/bin/library_setup.py
Cassandra: /Users/quachcd/mambaforge/envs/general/bin/cassandra.exe
Generating fragment libraries...


/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/formats/cassandramcf.py:117: UserWarning: Unable to infer coulombic 1-4 scaling factor. Setting to 0.0
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/formats/cassandramcf.py:154: UserWarning: Unable to infer LJ 1-4 scaling factor. Setting to 0.0
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([0.9899 2.1499 2.5635]), 0 bonds, id: 5928571952>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH2 pos=([1.009  2.0631 2.6893]), 0 bonds, id: 5928571520>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  

Running Cassandra...


CassandraRuntimeError: Cassandra exited with an error, see mosdef_cassandra_2022-06-27_15:42:49.785321.log for details.

### Exercise 6

In [ ]:
import numpy as np
import pylab as plt

data = np.genfromtxt("equil.out.prp", skip_header=3)

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r'MC sweeps')
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

ax.plot(data[:, 0], data[:,-1], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()